In [1]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import scipy.optimize as opt

In [2]:
VOTER0 = '../protein_classification_507_resnet34_hpv18.csv'
# VOTER0 = '../input/train.csv'
VOTER1 = '../protein_classification_512_resnet18_hpv18_cw.csv'
VOTER2 = '../protein_ensemble_0_513.csv'

In [3]:
def distribution(voters):
    count = np.zeros(28)
    for line in range(voters.shape[0]):
        temp = voters[line][1].split(' ')
        for tg in temp:
            if tg != '':
                count[int(tg)] += 1 
#                 print(tg, count[int(tg)])
    return count

In [4]:
voter0 = np.loadtxt(VOTER0, dtype=str, delimiter=',', comments=None)
voter1 = np.loadtxt(VOTER1, dtype=str, delimiter=',', comments=None)
voter2 = np.loadtxt(VOTER2, dtype=str, delimiter=',', comments=None)
# judge_count = (distribution(voter1)+distribution(voter2)+distribution(voter3))/3
# for i in range(judge_count.shape[0]):
#     if judge_count[i] < 200:
#         print(i)
poor_class_list = [8, 9, 10, 13, 15, 17, 20, 24, 27]

In [5]:
def hard_voting(voters, voters_num):
    count = np.zeros(28)
    for voters_index in range(voters_num):
        temp = voters[voters_index].split(' ')
        for tg in temp:
            if tg != '':
                count[int(tg)] += 1 
#                 print(tg, count[int(tg)])
    return count

In [6]:
voters = ["temp", "temp", "temp"]
submit = [["temp"] * 2] * (voter0.shape[0]+1)
submit[0] = ["Id", "Predicted"]
len(submit)

11704

In [7]:
for line in range(1, voter0.shape[0]):
    submit[line][0] = voter0[line][0]
    submit[line][1] = ""
    voters[0] = voter0[line][1]
    voters[1] = voter1[line][1]
    voters[2] = voter2[line][1]
    count=hard_voting(voters, 3)
    for i in range(count.shape[0]):
        if i in poor_class_list:
            if count[i] >= 1:
                submit[line][1] += str(i)+" "
        elif count[i] >= 2:
            submit[line][1] += str(i)+" "
    submit[line+1][1] = submit[line+1][1][:-1]
    voter0[line] = submit[line]
# len(submit)
# for line in range(len(submit)):
#     print(submit[line])
np.savetxt('hard-votted.csv', voter0, fmt='%s', delimiter=',')